In [8]:
# import sys
# sys.path.append("..")
from Environment.Env import Vertex, Graph, Agent_to_graph_assignment, cost_calculator
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Graph parameters

In [9]:
V = 8 # number of vertices
dropout_egde_rate = 0.4 # rate of edges to delete from the fully connected graph

# Create an undirected graph

## Create vertices

In [10]:
g = Graph()
for i in range(V):
    g.add(i,i)

# Set edges and build the adj matrix at the same time

In [11]:
adj_matrix = np.zeros((V,V))
for i in range(1,V):
    for j in range(i,V):
        if np.random.random() > dropout_egde_rate :
            adj_matrix[i,j], adj_matrix[j,i] = 1, 1
            g.addEdge(i,j,1)
            g.addEdge(j,i,1)

In [12]:
adj_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0., 0., 1.],
       [0., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 1., 0., 1., 1.],
       [0., 1., 1., 0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 1.]])

In [13]:
rows, cols = np.where(adj_matrix>0)
all_edges = {}
for row, col in zip(rows, cols):
    all_edges[(row, col)] = 1

In [14]:
all_edges

{(1, 1): 1,
 (1, 2): 1,
 (1, 4): 1,
 (1, 5): 1,
 (1, 6): 1,
 (1, 7): 1,
 (2, 1): 1,
 (2, 2): 1,
 (2, 3): 1,
 (2, 4): 1,
 (2, 6): 1,
 (3, 2): 1,
 (3, 4): 1,
 (3, 7): 1,
 (4, 1): 1,
 (4, 2): 1,
 (4, 3): 1,
 (4, 4): 1,
 (4, 5): 1,
 (4, 6): 1,
 (5, 1): 1,
 (5, 4): 1,
 (5, 6): 1,
 (5, 7): 1,
 (6, 1): 1,
 (6, 2): 1,
 (6, 4): 1,
 (6, 5): 1,
 (7, 1): 1,
 (7, 3): 1,
 (7, 5): 1,
 (7, 7): 1}

# Give names to the agents and randomly assign their starts and destinations

In [15]:
agents =["sto1",'adv1',"hyb1"]
assignement = Agent_to_graph_assignment(graph=g,
                                        list_agents_names = agents, adj_matrix=adj_matrix)
agents_dicts = assignement.random_assignement()

In [16]:
agents_dicts

[{'name': 'sto1',
  'infos': {'start': 2,
   'destination': 1,
   'arms': [{'path': [2, 1]},
    {'path': [2, 3, 4, 1]},
    {'path': [2, 3, 4, 5, 1]},
    {'path': [2, 3, 4, 5, 6, 1]},
    {'path': [2, 3, 4, 5, 7, 1]},
    {'path': [2, 3, 4, 6, 1]},
    {'path': [2, 3, 4, 6, 5, 1]},
    {'path': [2, 3, 4, 6, 5, 7, 1]},
    {'path': [2, 3, 7, 1]},
    {'path': [2, 3, 7, 5, 1]},
    {'path': [2, 3, 7, 5, 4, 1]},
    {'path': [2, 3, 7, 5, 4, 6, 1]},
    {'path': [2, 3, 7, 5, 6, 1]},
    {'path': [2, 3, 7, 5, 6, 4, 1]},
    {'path': [2, 4, 1]},
    {'path': [2, 4, 3, 7, 1]},
    {'path': [2, 4, 3, 7, 5, 1]},
    {'path': [2, 4, 3, 7, 5, 6, 1]},
    {'path': [2, 4, 5, 1]},
    {'path': [2, 4, 5, 6, 1]},
    {'path': [2, 4, 5, 7, 1]},
    {'path': [2, 4, 6, 1]},
    {'path': [2, 4, 6, 5, 1]},
    {'path': [2, 4, 6, 5, 7, 1]},
    {'path': [2, 6, 1]},
    {'path': [2, 6, 4, 1]},
    {'path': [2, 6, 4, 3, 7, 1]},
    {'path': [2, 6, 4, 3, 7, 5, 1]},
    {'path': [2, 6, 4, 5, 1]},
    {'path':

# Randomly choose an arm = succession of edges towards the arrival

Instead of randomly choosing, we will choose arms using the history of the costs using a Bandit algorithm. This is why we need to code the algorithms and adapt the code in `Agent_to_graph_assignment` to change to choice of arms according to the type of the agent

In [20]:
list_arms_pulled_ = [np.random.choice(agents_dicts[i]["infos"]['arms'])["path"] for i in range(len(agents_dicts))]
list_arms_pulled_

[[2, 4, 6, 5, 7, 1], [2, 3, 7, 5, 6, 4, 1], [6, 5, 4, 2, 1]]

# Compute calculations of costs

In [26]:
cc = cost_calculator(list_arms_pulled=list_arms_pulled_, 
                     adj_matrix= adj_matrix)
summary_round, history_costs_edges = cc.return_costs()

In [27]:
summary_round

{0: {'path': [2, 4, 6, 5, 7, 1], 'cost': 5.0},
 1: {'path': [2, 3, 7, 5, 6, 4, 1], 'cost': 6.0},
 2: {'path': [6, 5, 4, 2, 1], 'cost': 4.0}}

In [28]:
history_costs_edges

{(1, 1): [0, 0, 0, 0, 0, 0],
 (1, 2): [0, 0, 0, 1, 0, 0],
 (1, 4): [0, 0, 0, 0, 0, 1],
 (1, 5): [0, 0, 0, 0, 0, 0],
 (1, 6): [0, 0, 0, 0, 0, 0],
 (1, 7): [0, 0, 0, 0, 1, 0],
 (2, 1): [0, 0, 0, 1, 0, 0],
 (2, 2): [0, 0, 0, 0, 0, 0],
 (2, 3): [1, 0, 0, 0, 0, 0],
 (2, 4): [1, 0, 1, 0, 0, 0],
 (2, 6): [0, 0, 0, 0, 0, 0],
 (3, 2): [1, 0, 0, 0, 0, 0],
 (3, 4): [0, 0, 0, 0, 0, 0],
 (3, 7): [0, 1, 0, 0, 0, 0],
 (4, 1): [0, 0, 0, 0, 0, 1],
 (4, 2): [1, 0, 1, 0, 0, 0],
 (4, 3): [0, 0, 0, 0, 0, 0],
 (4, 4): [0, 0, 0, 0, 0, 0],
 (4, 5): [0, 1, 0, 0, 0, 0],
 (4, 6): [0, 1, 0, 0, 1, 0],
 (5, 1): [0, 0, 0, 0, 0, 0],
 (5, 4): [0, 1, 0, 0, 0, 0],
 (5, 6): [1, 0, 1, 1, 0, 0],
 (5, 7): [0, 0, 1, 1, 0, 0],
 (6, 1): [0, 0, 0, 0, 0, 0],
 (6, 2): [0, 0, 0, 0, 0, 0],
 (6, 4): [0, 1, 0, 0, 1, 0],
 (6, 5): [1, 0, 1, 1, 0, 0],
 (7, 1): [0, 0, 0, 0, 1, 0],
 (7, 3): [0, 1, 0, 0, 0, 0],
 (7, 5): [0, 0, 1, 1, 0, 0],
 (7, 7): [0, 0, 0, 0, 0, 0]}

# Compute Minimal cost and optimal paths

### Combinatorial approach : 

In [29]:
assignement.get_optimal_paths(combinatorial=True, time_limit=20)

Time depassed 20 seconds, only 37653 combinations where tested
Total time to compute costs:20.00 s
 => The minimal cost is :  4.0
 => The optimal paths are :  [[2, 1], [2, 4, 1], [6, 1]]
